# Extension of the Project for CS 401 Applied Data Analysis course

By: Team Data Nuage

Paper: [Link](https://cs.stanford.edu/people/jure/pubs/triads-chi10.pdf)

<hr style="border:5px solid gray"> </hr>

## Extension Description

### Title: Behaviour of edge signs in economic and non-living entities networks

#### Abstract

The analysis performed in this [work](https://cs.stanford.edu/people/jure/pubs/triads-chi10.pdf) enabled the authors to evaluate the social psychological theories of balance and status in social networks. This work allows us to better understand and predict the signs of relationships in social networks. The theories have been formulated on humans and were tested on them and the [paper](https://cs.stanford.edu/people/jure/pubs/triads-chi10.pdf) tests them on the connections formed by humans on social networking sites. To take this analysis one step further, we would like to observe how well these social theories hold up when the network context has changed. Moreover, we would also like to perceive the similarities and differences in analyzing the signs of relationships between entities created by people instead of the people themselves. For these two cases, we will be using a bitcoin dataset to provide a network in a financial setting (economic context) and a Reddit dataset that presents a graph of subreddits created by users. Our analysis would include observing and also presenting statistics about the number of balanced and unbalanced triads in the data and thereby interpreting the validity of balance and status theory and corroborate or contradicting the claims of the authors made in the original paper.

#### Research Questions

In this extension, we want to verify and discuss the generalization of the balance and status theory on other types of networks. Specifically, we want to answer the following questions

1. Whether the proposed theories on balance and status can be extended to `networks created based on monetary/profit relationships` **instead of** of `relationships created as social connections in slashdot data` i.e. will the relations between people (users) still follow the balance and status theory even when there is a direct cash transaction involved?
2. Secondly, will the theories of balance and status propagate from the **individual human relations** to the **nonliving entities** that are created and controlled by living entities? Here the graph is formed by the units that are created by humans and signs are based on relations between these units. Thus, we want to verify if the collective behavior of humans in those units follows the same laws as individuals. The datasets in the paper are representative of the direct opinion of one person on the other, unlike the Reddit dataset where the relation is between human-created units (that carry the opinion of humans).

#### Proposed datasets

For this extension, we are going to use the following two datasets.

In order to answer the first question we use [soc-sign-bitcoinalpha.csv.gz](http://snap.stanford.edu/data/soc-sign-bitcoin-alpha.html) dataset and use [soc-redditHyperlinks-body.tsv](http://snap.stanford.edu/data/soc-RedditHyperlinks.html) for the second question.

The signed network in the bitcoin transaction dataset is created by users trading Bitcoins, who rate other members on a scale of -10 (total distrust) to +10 (total trust) to prevent transactions with fraudulent and risky users.

The signed network in the Reddit dataset is the connections between two subreddits (a community on Reddit). Each connection represents the sentiments of one community towards the other.

#### Methods

##### Data Collection and Graph Building

We shall use the datasets from the above-provided URLs. As the datasets are already clean, we do not need to any preprocessing and build the graphs directly.

##### Undirected Analysis (to evaluate 'balance' theory)

1. Observe the counts of 4 possible signed triads.
2. Observe the frequency of triad occurrence with a random distribution of signs in the network.
3. Contrast between the original triad frequency and with random distribution.

##### Directed Analysis (to evaluate 'status' theory predictions)

1. Observe the frequency of Contextualized links (clink): It's a triple formed when node A and node B have an edge to or from another node X before A and B form an edge between each other.
2. Compute the generative baseline and surprise, receptive baseline, and surprise for each type of clink.
3. Using the above-computed metrics, check the consistency with Balance and Status theory.

##### Perform analysis on edge reciprocation

1. Observe the number of reciprocations for each type of edge that is the first connection being positive or negative and reciprocal connection being negative or positive
2. Calculate the probability of reciprocation for each type

In [1]:
# Importing Libraries
import os, pickle
import pandas as pd
import networkx as nx
from collections import Counter
from prettytable import PrettyTable
from tqdm import tqdm
import time
import numpy as np
import scipy as sp

<hr style="border:2px solid black"> </hr>

## Functions

Here we create functions that will be useful across all the datasets

#### Function to create a directed graph from the dataframe

In [2]:
def create_graph_from_df(dataframe_edges: pd.DataFrame):
    """
    This function accepts the edge list dataframe and returns a directed graph
    
    :param dataframe_edges (pd.DataFrame): A dataframe with edge lists of a graph
    :return rnd_graph (nx.Graph): A directed graph constructed using the edgelist
    """
    
    graph = nx.from_pandas_edgelist(dataframe_edges, source='FromNodeId', target='ToNodeId', edge_attr='Sign', create_using=nx.DiGraph())
    # create_using=nx.DiGraph() indicates to create a directed graph
    return graph

#### Function to count number of nodes and edges in a graph

In [3]:
def count_nodes_edges(graph: nx.Graph):
    """
    This function accepts the graph and returns the number of nodes and edges in the graph
    
    :param graph (nx.Graph): The graph to be counted
    :return nodes (int): The number of nodes in the graph
    :return edges (int): The number of edges in the graph
    """
    
    # Number of nodes in the graph
    nodes = graph.number_of_nodes()

    # Number of edges in the graph
    edges = graph.number_of_edges()
    
    return nodes, edges

#### Function to count number of positive and negative edges in a graph

In [4]:
def count_edge_signs(graph: nx.Graph):
    """
    This function accepts the graph and returns the percentage of positive and negative edges in the graph
    
    :param graph (nx.Graph): The graph to be counted
    :return positive (float): The percentage of positive edges in the graph
    :return negative (float): The percentage of negative edges in the graph
    """
    
    # we retrieve the edges of the graph to check the number of positive and negative links
    weights = graph.edges(data=True) 

    # weights is a list of tuples with the first node, second node, and edge attributes. We save the sign attribute to count the number of positive and negative edges.
    signs = [attr[2]["Sign"] for attr in weights]
    
    total = len(signs)

    # We calculate the number of positive and negative numbers 
    pos = len(list(filter(lambda x: (x >= 0), signs)))
    neg = len(list(filter(lambda x: (x < 0), signs)))
    
    # The percentage is calculated and rounded to one decimal place as to be consistent with the Table in the paper
    positive = round((pos/total)*100,1)
    negative = round((neg/total)*100,1)
    
    
    return positive, negative

#### Function to count the number of triads of one of the 4 types shown below in a graph

<img src="./images/figure_1.png" width=500 align="center" />

##### Logic for calculating the number of triads of one of the 4 types shown below in a graph

1. We convert the graph into an adjacency matrix along with the edge sign and consider the upper triangular part of it to avoid counting duplicate triads and the self-loops.
2. As the matrix is sparse and to avoid memory crash, we take the Coordinate list (COO) and Compressed sparse row (CSR) for zero diagonal matrices.
    * In COO format, instead of having access to all elements of the matrix, we only have access to the row, column, and the value of non zero elements by .row, .col, and .data commands.
3. We go over each node (primary node) of the graph using the upper triangular adjacency matrix (UTAM).
4. We get the list of nodes connected to the primary node and create a sub-square matrix from the CSR representation of the UTAM.
5. The submatrix contains all the adjacency representation of the nodes all nodes connected to the primary node and each nonzero entry of this submatrix represents a triad.
6. For each of the triad identified here, we get the sum of edge signs, and depending on the sum we increase the count of the respective triad. 
   
   
        A sum of 3 indicates a type T3, a sum of 1 indicates a type T2, a sum of -1 indicates a T1 and finally a sum of -3 indicates a T0.
        
**The reason for using a sparse matrix representation is to gain speed and not full out the available memory**
    
**As this an expensive calculation, we store the information of nodes and sum of signs in the pickle file and try to return that data if the function is called again. For this, a unique file name has to be provided.**    

In [5]:
def triad_tuple(graph: nx.Graph):
    """
    This function accepts the graph and returns the triads in the graph along with the sum of edge signs
    
    :param graph (nx.Graph): The graph to be counted
    :return triad_and_sum (list): A list with triad nodes and sum of edge signs
    """
    
    # a list to store the nodes of a triad and the sum of its signs
    triad_and_sum = list()

    #convert directed graph to undirected graph
    undir_graph = graph.to_undirected(reciprocal=False) # Reciprocal is set to false to retain edges that are unidirectional in the directed graph
    
    # calculating adjacency matrix using networkx
    adj_mat = nx.adjacency_matrix(undir_graph, weight='Sign')
    
    # upper triangle adjacency matrix in coo format
    UA = sp.sparse.triu(adj_mat, k=1)
    # upper triangle adjacency matrix in csr format
    UA_csr = UA.tocsr()
    
    # total number of all the nodes of the network
    n_nodes = UA.shape[0]
    
    for prim_node in range(n_nodes):
        idx = (UA.row == prim_node)

        # possible triad nodes to the primary node
        pos_nodes = UA.col[idx]

        # sub matrix
        conn_submatrix = UA_csr[pos_nodes, pos_nodes.reshape(-1, 1)]

        # the values of the nodes of each triad:
        vertex_1 = pos_nodes[conn_submatrix.tocoo().row].reshape(-1, 1)
        vertex_2 = pos_nodes[conn_submatrix.tocoo().col].reshape(-1, 1)
        vertex_3 = (prim_node * np.ones(len(vertex_1))).reshape(-1, 1)
        
        possible_triads = np.concatenate([vertex_3, vertex_1, vertex_2], axis=1).astype('int')
        for node_1, node_2, node_3 in possible_triads:
            tot_sum = UA_csr[node_1, node_2] + UA_csr[node_3, node_2] + UA_csr[node_1, node_3]
            triad_and_sum.append(((node_1, node_2, node_3), tot_sum))
    
    return triad_and_sum


def count_undir_triads(graph: nx.Graph, file_path: str):
    """
    This function accepts the graph and returns the number of triads in the graph for each type in an undirected setting
    
    :param graph (nx.Graph): The graph to be counted
    :param file_path (str): The path of the file to store/load the triad nodes in a pickle (binary) file.
    :return triad_counts (dict): The number of triads of each type in the graph
    """
    
    
    # empty dictionary to store triad counts
    triad_counts = {"T3":0, "T2":0, "T1":0, "T0":0}

    # a dictionry to indicate the type of triad based on the sum of signs
    triad_type = {3:"T3", 1:"T2", -1:"T1", -3:"T0"}

    # checking if triad list is already available
    try:
        triad_and_sum = pickle.load(open(file_path+".pickle", "rb"))
    
    except IOError:
        triad_and_sum = triad_tuple(graph) 
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path+".pickle", 'wb') as handle:
            pickle.dump(triad_and_sum, handle, protocol=pickle.HIGHEST_PROTOCOL)

    summed_signs = [trisum[1] for trisum in triad_and_sum]
    sum_count = Counter(summed_signs)
    triad_counts = {triad_type[sumval]:count for sumval, count in sum_count.items()}

    return triad_counts

#### Function to randomise signs in the dataframe according to the proportion of positive and negative signs

In [6]:
def randomize_df(graph_df: pd.DataFrame):
    """
    This function accepts the edge list data frame and returns the data frame containing the edges with random sign distribution.
    
    :param graph_df (pd.DataFrame): A dataframe with edge lists of a graph
    :return rnd_graph_df (pd.DataFrame): the dataframe with random sign edges
    """
    
    rnd_graph_df = graph_df.copy()
    rnd_graph_df = rnd_graph_df.sample(frac=1) # shuffle
    rnd_graph_df.reset_index(inplace=True, drop=True)
    rnd_graph_df['FromNodeId'] = graph_df['FromNodeId']
    rnd_graph_df['ToNodeId'] = graph_df['ToNodeId']
    rnd_graph_df["Time"] = graph_df['Time']

    return rnd_graph_df

#### Function to calculate the statistics related analysis of undirected graph.

In [7]:
def get_undir_surprise(Ti: int, PoTi: float, delta: int):
    """
    This function accepts the number of triads of a type, prior probability of that of the triad, and the total number of triads and
    returns the surprise.
    
    :param Ti (int): The number of triads of a type in the graph
    :param PoTi (float):  A priori probability of the type of triad
    :param delta (int): The total number of triads in the graph
    :return surp (float): The surprise which is defined as the number of standard deviations by which the actual quantity of triads of a type differs
    from the expected number under the random-shuffling model.
    """
    ETi = PoTi * delta
    sup_num = Ti - ETi
    sup_den = np.sqrt(ETi*(1-PoTi))
    surp = sup_num/sup_den
    return surp


def get_undirected_metrics(graph_udir_triads: dict, rnd_graph_udir_triads: dict):
    """
    This function accepts the triad count of the graph and the random graph and
    returns the Fraction of triads of each type along with the prior probability and the surprise.
    
    :param graph_udir_triads (dict): The number of triads of each type in the graph
    :param rnd_graph_udir_triads (dict): The number of triads of each type in the graph
    :return graph_stats (dict): A dictionary containing the information on the number of balanced and unbalanced undirected triads in the graph
    """
    
    graph_stats = {}
    
    delta = sum(graph_udir_triads.values())
    graph_stats["delta"] = delta
    
    for ttype, Ti in graph_udir_triads.items():
        
        pTi = Ti/delta
        poTi = rnd_graph_udir_triads[ttype]/delta
        STi = get_undir_surprise(Ti, poTi, delta)
        
        graph_stats[ttype] = {"Ti": Ti, "pTi":round(pTi, 3), "poTi":round(poTi, 3), "STi": round(STi,1)}
    
    return graph_stats

#### Function to calculate the generative and receptive baseline of each node.

In [8]:
def get_baselines(graph:nx.DiGraph):
    """
    This function accepts the directed graph and returns the generative and receptive baselines for each node.
    
    :param graph (nx.DiGraph): The directed graph
    :return gen_base (dict): A dictionary with keys as node and the values as their generative baselines.
    :return rec_base (dict): A dictionary with keys as node and the values as their receptive baselines.
    """
    
    # dictionaries to store baselines
    gen_base = {}
    rec_base = {}
    
    # for each node
    for node in tqdm(graph):
        
        # get the incoming edges and calculate the generative baseline
        try:
            inedges = graph.in_edges(node, data=True)
            gen_num = sum(in_edge[2]["Sign"] for in_edge in inedges if in_edge[2]["Sign"] == 1)
            gen_base[node] = gen_num/len(inedges)
        except:
            gen_base[node] =0
        
        # get the outgoing edges and calculate the recptive baseline
        try:
            outedges = graph.out_edges(node, data=True)
            rec_num = sum(out_edge[2]["Sign"] for out_edge in outedges if out_edge[2]["Sign"] == 1)
            rec_base[node] = rec_num/len(outedges)
        except:
            rec_base[node] = 0
            
    return gen_base, rec_base

#### Function to statistics related to directed graph

* To analyze the network as it grows, we do not build the entire graph at once, we build edge by edge based on the time they were created.

    1. We sort the data frame based on the `Time` column.
    2. We go over each edge in the data frame and get the common neighbors between the from node and to node.
    3. Then, get the type of edges between the to and from node to the common neighbor and update the respective count, generative, and receptive base.
    4. Using the edge sign we assign the status to from node and to node and subsequently append it to the triad type.
    5. We also append the sign prediction using balance theory to evaluate the balance theory.

##### Prediction of edge sign using Balance theory

* According to balance theory triads with three positive signs and one positive sign are more popular than others. Using this we predict as follows
    1. If the two edges already existing are positive, then the third edge would be positive.
    2. If the two edges already existing are negative, then the third edge would be positive.
    3. If there is one positive edge, one negative edge already existing, then the third edge would be negative.

        
This will make the prediction of edge as positive for triads of type $t_1$, $t_3$, $t_6$, $t_8$, $t_9$, $t_{11}$, $t_{14}$ and $t_{16}$, and negative for the remaning.
   

##### Edge Reciprocation

* To analyze the reciprocal relationships, we examine the probability of nodes exchanging signs. There are four possibilities for reciprocation which is given a positive (or a negative) edge from node A to node B there can a positive or a negative (negative or a positive) edge from node B to A.

* For calculation, while adding a new edge to the graph we check if a reverse edge exists and if so, increase the count.


In [9]:
def get_directed_stats(edge_dataframe: pd.DataFrame, tri_mapping: dict, gen_base: dict, rec_base: dict):  
    """
    This function accepts the data frame containing the edges and the Time of edge creation,
    the mapping of triad types, generative and receptive bases of nodes and
    return the census for each type of the 16 triads in a directed graph.
    The census includes the number of triads of the specified type,
    the number of positive edges created while completing the triad (i.e. A-B edges),
    the generative and receptive base of the triad type,
    the status of A and B in each of the triad in each triad type. 
    Along with the census, a dictionary containing the count for each type of reciprocation as mentioned above.
    
    :param edge_dataframe (pd.DataFrame): The data frame with edge list
    :param tri_mapping (dict): A dictionary mapping the triad edge attribute to its type.
    :param gen_base (dict): A dictionary with keys as a node and the values as their generative baselines.
    :param rec_base (dict): A dictionary with keys as a node and the values as their receptive baselines.
    :return dir_graph_stats (dict): A dictionary of dictionary with triad type as first-level key and
    the count, positive edges, generative base, receptive base, statuses of from node, statuses of to node,
    the edge sign predicted using balance theory.
    :return recpr_dict (dict): A dictionary with key as the type of reciprocation and value as the count
    """
    
    # an empty directed graph to store the building graph 
    dir_graph = nx.DiGraph()
    
    sorted_dataframe = edge_dataframe.sort_values(by='Time', ascending=True).reset_index(drop=True)
    
    # a dictionary to store the information related to each type of 16 triads
    dir_graph_stats = {"t"+str(i):{"tot":0, "pos":0, "gb":0,"rb":0, "status_a": 0, "status_b": 0, "bal_pred_sign": 0} for i in range(1,17)}
    # a dictionary to store the count related to each type of reciprocation
    recpr_dict = {"P(+|+)":0,"P(-|+)":0,"P(+|-)":0,"P(-|-)":0}
    
    # building the graph
    for rid, row in tqdm(sorted_dataframe.iterrows()):
        frm_node = row["FromNodeId"]
        to_node = row["ToNodeId"]
        sgn = row["Sign"]
        
        # check if the inverse edge exists and increase the corresponding count
        if dir_graph.has_edge(to_node, frm_node):
            fst_sgn = dir_graph.get_edge_data(to_node, frm_node)["Sign"]
            
            sgn_pair = (sgn, fst_sgn)
            if sgn_pair == (1,1):
                recpr_dict["P(+|+)"] += 1
            elif sgn_pair == (-1,1):
                recpr_dict["P(-|+)"] += 1
            elif sgn_pair == (1,-1):
                recpr_dict["P(+|-)"] += 1
            elif sgn_pair == (-1,-1):
                recpr_dict["P(-|-)"] += 1
            
        
        # check if the from node and to node are already in the graph to check for triads
        if (frm_node in dir_graph) and (to_node in dir_graph) and (frm_node != to_node):
            
            # get the neighbors of from node and the to node
            frm_nbs = set(dir_graph.successors(frm_node)).union(dir_graph.predecessors(frm_node))
            to_nbs = set(dir_graph.successors(to_node)).union(dir_graph.predecessors(to_node))
            # get the common neighbors between the from node and to node
            cmn_nbs = set(frm_nbs.intersection(to_nbs))
            
            for nbr in cmn_nbs:
                # check and ignore self loops for from node and to node
                if nbr not in [frm_node, to_node]:
                    # the two lists stores the direction and sign of the connection between common neighbour and from and to nodes.
                    frm_nbr = []
                    to_nbr = []

                    # check if the connection is originating from the from (to) node or neighbour
                    if dir_graph.has_edge(frm_node, nbr):
                        frm_nbr.append(("fs", dir_graph.get_edge_data(frm_node, nbr)["Sign"]))
                    if dir_graph.has_edge(nbr, frm_node):
                        frm_nbr.append(("fr", dir_graph.get_edge_data(nbr, frm_node)["Sign"]))

                    if dir_graph.has_edge(to_node, nbr):
                        to_nbr.append(("ts", dir_graph.get_edge_data(to_node, nbr)["Sign"]))
                    if dir_graph.has_edge(nbr, to_node):
                        to_nbr.append(("tr", dir_graph.get_edge_data(nbr, to_node)["Sign"]))

                    for from_type in frm_nbr:
                        for to_type in to_nbr:
                            # get the type of triad
                            tri_type = tri_mapping[frozenset([from_type, to_type])]
                            # increase the count of the triad type
                            dir_graph_stats[tri_type]["tot"] += 1 
                            # if the sign of the to be formed link is positive, increase the count of positive links for the type of c-link
                            if sgn == 1:
                                dir_graph_stats[tri_type]["pos"] += 1
                            # add the generative (receptive) base line of from (to) node to the triad type 
                            dir_graph_stats[tri_type]["gb"] += gen_base[frm_node]
                            dir_graph_stats[tri_type]["rb"] += rec_base[to_node]

                            # status of from node
                            if from_type in [("fr",1), ("fs",-1)]:
                                dir_graph_stats[tri_type]["status_a"] += 1
                            else:
                                dir_graph_stats[tri_type]["status_a"] += -1

                            # status of to node
                            if to_type in [("tr",1), ("ts",-1)]:
                                dir_graph_stats[tri_type]["status_b"] += 1
                            else:
                                dir_graph_stats[tri_type]["status_b"] += -1
                            
                            # prediction using balance theory
                            if tri_type in ["t1", "t3", "t6", "t8", "t9", "t11", "t14", "t16"]:
                                dir_graph_stats[tri_type]["bal_pred_sign"] += 1
                            else:
                                dir_graph_stats[tri_type]["bal_pred_sign"] += -1
                                
        dir_graph.add_edge(frm_node, to_node, Sign=sgn)
    return dir_graph_stats, recpr_dict

#### Function to calculate the probability of a positive edge in directed graph.

We calculate the probability of positive edges by dividing the number of positive edges by total number of edges.

In [10]:
def get_posprob(dir_graph_stats: dict):
    """
    This function accepts the stats related to a directed graph and returns the probability of positive edges
    
    :param dir_graph_stats (dict): A dictionary of dictionary with triad type as first level key and
    the count, positive edges, generative base, receptive base, statuses of from node, statuses of to node.
    :return pos_prob (dict): A dictionary with keys as triad type and the values as probability of positive edge in the triad.
    """
    
    pos_prob = {ttype: dir_graph_stats[ttype]["pos"]/dir_graph_stats[ttype]["tot"] for ttype in dir_graph_stats}
    return pos_prob

#### Function to calculate the generative and receptive surprise in each type of triad and respective evaluation of Balance and Status theories

We shall follow a similar procedure used for undirected graph except that here the calculation is for the edges.

In [11]:
def directed_evaluation(dir_graph_stats: dict, pos_prob: dict):
    """
    This function accepts the stats related to a directed graph, the apriori probability of having a positive edge
    and return the probability of having a positive edge in a triad, generative surprise, and receptive surprise.
    
    :param dir_graph_stats (dict): A dictionary of dictionary with triad type as first level key and
    the count, positive edges, generative base, receptive base, statuses of from node, statuses of to node.
    :param pos_prob (dict): A dictionary with keys as triad type and the values as probability of positive edge in the triad.
    :return dir_graph_surp (dict): A dictionary of dictionary with triad type as first level key and
    the probability of positive edges, generative surprise, receptive surprise, booleans indicating
    consistency of balance with generative surprise,
    consistency of balance with receptive surprise,
    consistency of status with generative surprise,
    consistency of status with receptive surprise.
    """
    
    def get_surp(actual, expected, p_prob):
        sup_num = actual - expected
        sup_den = np.sqrt(expected * (1 - p_prob))
        surp = sup_num/sup_den
        return surp
    
    dir_graph_surp = {}
    for ttype, tprop in dir_graph_stats.items():
        dir_graph_surp[ttype] = {}
        dir_graph_surp[ttype]["count"] = tprop["tot"]
        dir_graph_surp[ttype]["P+"] = round(tprop["pos"]/tprop["tot"], 2)
        dir_graph_surp[ttype]["Po+"] = round(pos_prob[ttype], 2)
        
        # gen_surp denotes the generative surprise
        gen_surp = get_surp(tprop["pos"], tprop["gb"], pos_prob[ttype])
        dir_graph_surp[ttype]["gen_surp"] = round(gen_surp, 1)
        
        # consistency of status with generative surprise
        # in case of equal positive and negative status we assume the sign matches
        if np.sign(tprop["status_b"]) in [0, np.sign(gen_surp)]:
            sg = True
        else:
            sg = False
        
        dir_graph_surp[ttype]["sg"] = sg
        
        # consistency of balance with generative surprise
        # in case of equal positive and negative predictions we assume the prediction matches
        if np.sign(tprop["bal_pred_sign"]) in [0, np.sign(gen_surp)]:
            bg = True
        else:
            bg = False
            
        dir_graph_surp[ttype]["bg"] = bg
            
        # rec_surp denotes the receptive surprise
        rec_surp = get_surp(tprop["pos"], tprop["rb"], pos_prob[ttype])
        dir_graph_surp[ttype]["rec_surp"] = round(rec_surp, 1)
        
        # consistency of status with receptive surprise,
        # in case of equal positive and negative status we assume the does not sign matches
        if np.sign(tprop["status_a"]) not in [np.sign(rec_surp)]:
            sr = True
        else:
            sr = False
        
        dir_graph_surp[ttype]["sr"] = sr
        
        # consistency of balance with receptive surprise,
        # in case of equal positive and negative predictions we assume the prediction matches
        if np.sign(tprop["bal_pred_sign"]) in [0, np.sign(rec_surp)]:
            br = True
        else:
            br = False
        
        dir_graph_surp[ttype]["br"] = br
        
    return dir_graph_surp

#### Function to draw tables

In [12]:
def draw_stats_table(nu_nodes: int, nu_edges: int, frac_pos_edges: float, frac_neg_edges: float, triad_count):
    stats_table = PrettyTable()
    stats_table.field_names = [" ", "Value"]
    stats_table.add_row(["Nodes", nu_nodes])
    stats_table.add_row(["Edges", nu_edges])
    stats_table.add_row(["+Edges %", frac_pos_edges])
    stats_table.add_row(["-Edges %", frac_neg_edges])
    stats_table.add_row(["Triads",  triad_count])
    return stats_table

def draw_undir_table(undir_graph_stats:dict):
    undir_table = PrettyTable()
    undir_table.field_names = ["Triad Ti ", "|Ti|", "P(Ti)", "Po(Ti)", "S(Ti)"]
    undir_table.add_row(["T3 | + + +", undir_graph_stats["T3"]["Ti"], undir_graph_stats["T3"]["pTi"], undir_graph_stats["T3"]["poTi"], undir_graph_stats["T3"]["STi"]])
    undir_table.add_row(["T1 | + - -", undir_graph_stats["T1"]["Ti"], undir_graph_stats["T1"]["pTi"], undir_graph_stats["T1"]["poTi"], undir_graph_stats["T1"]["STi"]])
    undir_table.add_row(["T2 | + + -", undir_graph_stats["T2"]["Ti"], undir_graph_stats["T2"]["pTi"], undir_graph_stats["T2"]["poTi"], undir_graph_stats["T2"]["STi"]])
    undir_table.add_row(["T0 | - - -", undir_graph_stats["T0"]["Ti"], undir_graph_stats["T0"]["pTi"], undir_graph_stats["T0"]["poTi"], undir_graph_stats["T0"]["STi"]])
    return undir_table

def draw_dir_table(dir_graph_stats:dict):
    dir_table = PrettyTable()
    dir_table.field_names = ["ti", "count", "P(+)", "sg", "sr", "Bg", "Br", "Sg", "Sr"]
    for tri_type, tri_stats in dir_graph_stats.items():
        dir_table.add_row([tri_type, tri_stats["count"], tri_stats["P+"], tri_stats["gen_surp"], tri_stats["rec_surp"], tri_stats["bg"], tri_stats["br"], tri_stats["sg"], tri_stats["sr"]])
    return dir_table

def draw_recp_table(recpr_dict: dict):
    
    tot_recp_pos = sum(val[1] for val in recpr_dict.items() if val[0] in ["P(+|+)", "P(-|+)"])
    tot_recp_neg = sum(val[1] for val in recpr_dict.items() if val[0] in ["P(+|-)", "P(-|-)"])
    
    recp_table = PrettyTable()
    recp_table.field_names = ["Reciprocation", "count", "Fraction"]
    for recp_type, recp_stats in recpr_dict.items():
        cnt = recpr_dict[recp_type]
        if recp_type in ["P(+|+)", "P(-|+)"]:
            recp_table.add_row([recp_type, cnt, round(cnt/tot_recp_pos, 2)])
        else:
            recp_table.add_row([recp_type, cnt, round(cnt/tot_recp_neg, 2)])
    return recp_table

For the analysis of the network as a directed graph, we need to count the 16 types of triads as shown in the figure below.


<img src="./images/figure_2.png" width=350 align="center" />

We call the `A` node as from node, `B` node as to node (as the triad is generated by forming the edge from `A` to `B`), and `X` as a common neighbor. If the `from node` is receiving a connection from the common neighbor, we shall name it "fr" and "fs" in the opposite case. Similarly, if the `to node` is receiving a connection from a common neighbor, we shall name it "tr" and "ts" in the opposite case.

Below we store this mapping in a dictionary. As each triad is represented by two attributes, we will use a frozen set of the from attribute and to attribute as the key of the dictionary. The reason for using a frozen set is it is hashable because it's immutable.

In [13]:
tri_mapping = {frozenset([("fs",1),("tr",1)]):"t1",
              frozenset([("fs",1),("tr",-1)]):"t2",
              frozenset([("fs",1),("ts",1)]):"t3",
              frozenset([("fs",1),("ts",-1)]):"t4",
              frozenset([("fs",-1),("tr",1)]):"t5",
              frozenset([("fs",-1),("tr",-1)]):"t6",
              frozenset([("fs",-1),("ts",1)]):"t7",
              frozenset([("fs",-1),("ts",-1)]):"t8",
              frozenset([("fr",1),("tr",1)]):"t9",
              frozenset([("fr",1),("tr",-1)]):"t10",
              frozenset([("fr",1),("ts",1)]):"t11",
              frozenset([("fr",1),("ts",-1)]):"t12",
              frozenset([("fr",-1),("tr",1)]):"t13",
              frozenset([("fr",-1),("tr",-1)]):"t14",
              frozenset([("fr",-1),("ts",1)]):"t15",
              frozenset([("fr",-1),("ts",-1)]):"t16"}

<hr style="border:2px solid black"> </hr>

## Analysis

### BitCoin Alpha Dataset

#### Dataset Description

The signed network in the bitcoin transaction dataset is created by users trading Bitcoins, who rate other members on a scale of -10 (total distrust) to +10 (total trust) to prevent transactions with fraudulent and risky users. The data is stored in a compressed CSV file with no headers. There are four columns in the dataset and they are the source node, the destination node, the trust value, and the time of the creation of the edge.

**As our analysis relates to the sign of the edge rather than the numerical value, we consider all the trust values less than 0 are -1 and the remaining as +1.**

#### Creating the graph

As the CSV file does not have any header, we skip the header and provide names to the columns and create a directed graph using the data frame as an edge list.

In [14]:
def get_sign(weight):
    if weight < 0:
        return -1
    return 1

## read the data to the dataframe
bitcoin_df = pd.read_csv("./data/soc-sign-bitcoinalpha.csv.gz", header=None, names=["FromNodeId", "ToNodeId", "Sign", "Time"])
bitcoin_df['Sign'] = bitcoin_df['Sign'].map(get_sign)

bitcoin_graph = create_graph_from_df(bitcoin_df)

#### Dataset Statistics

In [15]:
## Counting Nodes and Edges in Bitcoin Alpha network
bitcoin_nodes, bitcoin_edges = count_nodes_edges(bitcoin_graph)
print("The number of nodes in Bitcoin Alpha network are {0}".format(bitcoin_nodes))
print("The number of edges in Bitcoin Alpha network are {0}".format(bitcoin_edges))
print()
## Counting the signs of the Edges in Bitcoin Alpha network
bitcoin_pos_edges, bitcoin_neg_edges = count_edge_signs(bitcoin_graph)
print("The percentage of positive edges in the Bitcoin Alpha network is {0}".format(bitcoin_pos_edges))
print("The percentage of negative edges in the Bitcoin Alpha network is {0}".format(bitcoin_neg_edges))
print()

The number of nodes in Bitcoin Alpha network are 3783
The number of edges in Bitcoin Alpha network are 24186

The percentage of positive edges in the Bitcoin Alpha network is 93.6
The percentage of negative edges in the Bitcoin Alpha network is 6.4



#### Analysis of the Bitcoin network as an undirected network

In [16]:
# Counting the triads of each type in undirected Bitcoin Alpha network
bitcoin_undir_triads = count_undir_triads(bitcoin_graph, "./data/cache/bitcoinalpha_undir_triad")

# creating a Bitcoin Alpha graph with randomised edge signs
rnd_bitcoin_graph_df = randomize_df(bitcoin_df)
rnd_bitcoin_graph = create_graph_from_df(rnd_bitcoin_graph_df)

# Counting the triads of each type in undirected Bitcoin Alpha network with randomised edge signs
rnd_bitcoin_undir_triads = count_undir_triads(rnd_bitcoin_graph, "./data/cache/rnd_bitcoinalpha_undir_triad")

bitcoin_graph_stats = get_undirected_metrics(bitcoin_undir_triads, rnd_bitcoin_undir_triads)

print("The number of Triads in Bitcoin Alpha network are {0}".format(bitcoin_graph_stats["delta"]))

The number of Triads in Bitcoin Alpha network are 22153


#### Analysis of the Bitcoin network as an evolving directed network

Using the already created graph, we obtain the generative and recptive baselines for each node.

In [17]:
btc_genbase, btc_recbase = get_baselines(bitcoin_graph)

100%|██████████| 3783/3783 [00:00<00:00, 56459.75it/s]


Get the census realted to the directed graph

In [18]:
btc_dirgraph_stats, bitcoin_recpr_dict = get_directed_stats(bitcoin_df, tri_mapping, btc_genbase, btc_recbase)

24186it [00:02, 11191.76it/s]


To calculate the surprise i.e. the number of standard deviations by which the actual number of positive A-B edges for a triad differs above or below the number obtained through the bases we need the prior probability of having a positive edge for each triad type. For this, we shall keep the edge sequence and order the same and randomize the edge sign while maintaining the number of positive and negative signs. This is already obtained while performing the analysis on the undirected graph and we shall the same.

In [19]:
# Get the census realted to the randomised directed graph
# As we only need the number of positive edges for each type of triad and do not bother about other attributes, we use the ols generative and receptive bases
btc_rnd_dirgraph_stats, _ = get_directed_stats(rnd_bitcoin_graph_df, tri_mapping, btc_genbase, btc_recbase)

24186it [00:02, 11522.64it/s]


Using the information on number of positive edges in each type of triad, obtained by randomised sign graph. We get the priori probability of having a positive edge for each triad type.

In [20]:
btc_PoTi = get_posprob(btc_rnd_dirgraph_stats)

# Evaluation with respect to balance and status theories.
btc_eval = directed_evaluation(btc_dirgraph_stats, btc_PoTi)

#### Visualisation and Discussion

In [21]:
print("Bitcoin Alpha Dataset Statitics on undirected graph")
print(draw_stats_table(bitcoin_nodes, bitcoin_edges, bitcoin_pos_edges, bitcoin_neg_edges, bitcoin_graph_stats["delta"]))
print("\n")
print("Analysis of Undirected Bitcoin Alpha Dataset")
print(draw_undir_table(bitcoin_graph_stats))

Bitcoin Alpha Dataset Statitics on undirected graph
+----------+-------+
|          | Value |
+----------+-------+
|  Nodes   |  3783 |
|  Edges   | 24186 |
| +Edges % |  93.6 |
| -Edges % |  6.4  |
|  Triads  | 22153 |
+----------+-------+


Analysis of Undirected Bitcoin Alpha Dataset
+------------+-------+-------+--------+-------+
| Triad Ti   |  |Ti| | P(Ti) | Po(Ti) | S(Ti) |
+------------+-------+-------+--------+-------+
| T3 | + + + | 17014 | 0.768 | 0.825  | -22.4 |
| T1 | + - - |  1667 | 0.075 | 0.011  |  93.4 |
| T2 | + + - |  3333 |  0.15 | 0.164  |  -5.4 |
| T0 | - - - |  139  | 0.006 |  0.0   |  46.3 |
+------------+-------+-------+--------+-------+


##### Description of undirected analysis results

The triads for this network can be interpreted as below
- $T_3$: If two users trust a common user then they trust each other.
- $T_1$: If two users distrust a common user then they trust each other.
- $T_2$: If two users trust a common user then they distrust each other.
- $T_0$: If two users distrust a common user then they distrust each other.

We find that the all-positive triad $T_3$, two-positive triad $T_2$ are underrepresented, and the triads of type $T_1$ and $T_0$ are overrepresented. This phenomenon cannot be explained by both Heider’s original notion of structural balance and Davis’s weaker notion of balance. 

<hr style="border:1px solid black"> </hr>

In [22]:
print("ti incidates one of the 16 type of triads shown above, P(+) and Po(+) are the probability and a prori probability that the closing red edge is positive.")
print("sg: surprise of edge initiator giving a positive edge; sr: surprise of edge destination receiving a positive edge") 
print("Bg: consistency of balance with generative surprise; Br: consistency of balance with receptive surprise; \nSg: consistency of status with generative surprise; Sr: consistency of status with receptive surprise.\n")
print("Analysis of the Bitcoin network as an evolving directed network\n")
print(draw_dir_table(btc_eval))

ti incidates one of the 16 type of triads shown above, P(+) and Po(+) are the probability and a prori probability that the closing red edge is positive.
sg: surprise of edge initiator giving a positive edge; sr: surprise of edge destination receiving a positive edge
Bg: consistency of balance with generative surprise; Br: consistency of balance with receptive surprise; 
Sg: consistency of status with generative surprise; Sr: consistency of status with receptive surprise.

Analysis of the Bitcoin network as an evolving directed network

+-----+-------+------+-------+-------+-------+-------+-------+-------+
|  ti | count | P(+) |   sg  |   sr  |   Bg  |   Br  |   Sg  |   Sr  |
+-----+-------+------+-------+-------+-------+-------+-------+-------+
|  t1 | 26910 | 0.92 | -23.5 |  6.0  | False |  True | False |  True |
|  t2 |  1105 | 0.22 | -89.4 | -82.5 |  True |  True |  True | False |
|  t3 | 27368 | 0.9  | -37.2 | -15.3 | False | False |  True | False |
|  t4 |  709  | 0.67 | -21.2 | -

##### Description of directed analysis results

In the Bitcoin Alpha dataset, triad type t1, t3, t9, and t11 are abundant. All these triads have positive edges although the directions differ. These triads with all positive directed edges are only slightly underrepresented.

We see that the directed triads with one positive directed link in either direction and the other link being negative are  underrepresented. To continue with this observation the other type of triads are also underrepresented. However, we can observe that the triads with all positive directed edges are only slightly underrepresented.

Out of the 16 possible triads, the predictions of balance performs better than those of the status. The balance theory is consistent on 10 types with respect to generative surprise where status theory is only consistent in 8 types. With regard to the receptive surprise, the balance theory is consistent with 12 types and the status theory is consistent only on 8 types. While a look at both generative and receptive surprise together, balance theory is consistent in 9 types where are the status theory is consistent only in 5 types. 

<hr style="border:1px solid black"> </hr>

#### Analysis of the Reciprocity in the Bitcoin network.

In [23]:
print("Analysis of edge Reciprocation in directed Bitcoin Alpha Dataset")
print(draw_recp_table(bitcoin_recpr_dict))
tot_recp = sum(val for val in bitcoin_recpr_dict.values())
btc_recp_perc = (tot_recp/bitcoin_edges)*100
print("The percentage of edges that reciprocate the edge are {0:.2f}".format(btc_recp_perc))

Analysis of edge Reciprocation in directed Bitcoin Alpha Dataset
+---------------+-------+----------+
| Reciprocation | count | Fraction |
+---------------+-------+----------+
|     P(+|+)    |  9678 |   0.98   |
|     P(-|+)    |  222  |   0.02   |
|     P(+|-)    |   26  |   0.16   |
|     P(-|-)    |  136  |   0.84   |
+---------------+-------+----------+
The percentage of edges that reciprocate the edge are 41.60


##### Description of on Edge Reciprocity results

##### TODO: Nikhil

<hr style="border:2px solid black"> </hr>

### Reddit Hyperlink Network Dataset

#### Dataset Description

The signed network in the Reddit dataset is the connections between two subreddits (a community on Reddit). Each connection represents the sentiments of one community towards the other in one post. Thus, there will be many connections from one subreddit to another. **We merge all the edges from one subreddit to another and take the majority sign as the sign of the edge between them.** The data is stored in a tsv file with `SOURCE_SUBREDDIT, TARGET_SUBREDDIT, POST_ID, TIMESTAMP, LINK_SENTIMENT, PROPERTIES` as headers. We ignore the POST_ID and PROPERTIES as we deal only with the sign of the edge. The *SOURCE_SUBREDDIT* will be the from node, *TARGET_SUBREDDIT* will be the to node, *LINK_SENTIMENT* which is 1 or -1 will be the edge sign.

#### Creating the graph

As the tsv file has a header, we skip the columns we do not need and create the data frame.

Firstly, We store the information about the edges in a dictionary of dictionaries. The first level key is the from node, the second level key will be the to node and the values are the sign of the edge and the time stamp of the last created link.

Secondly, we store this dataframe in a binay file to avoid the computation.

Lastly, We create a dictionary to store the subreddit name mapped to a unique integer as to avoid any potential memory issues while using networkx.

In [24]:
# try reading the pickle file. Users should chnage the bin_file variable to the address of the file


bin_file = "./data/subreddit_df"

try:
    reddit_df = df = pd.read_pickle(bin_file)
except IOError:
    # read the data to the dataframe
    reddit_df_full = pd.read_csv("./data/soc-redditHyperlinks-body.tsv.gz", sep="\t", usecols=["SOURCE_SUBREDDIT", "TARGET_SUBREDDIT", "TIMESTAMP", "LINK_SENTIMENT"])

    # empty dictionary to store the network
    reddit_adj_list = {}

    for _, edge in reddit_df_full.iterrows():
        from_node = edge["SOURCE_SUBREDDIT"]
        to_node = edge["TARGET_SUBREDDIT"]
        if from_node not in reddit_adj_list:
            reddit_adj_list[from_node] = {}
        if to_node not in reddit_adj_list[from_node]:
            reddit_adj_list[from_node][to_node] = {"Sign": 0, "Time":None}
        reddit_adj_list[from_node][to_node]["Sign"] += edge["LINK_SENTIMENT"]
        reddit_adj_list[from_node][to_node]["Time"] = edge["TIMESTAMP"]

    # convert the dictionary back into dataframe to reuse the function created.
    reddit_list = []
    for from_node, to_nodes in reddit_adj_list.items():
        for to_node, edges in to_nodes.items():
            reddit_list.append([from_node, to_node, get_sign(edges["Sign"]), edges["Time"]])

    col_names = ["FromNodeId", "ToNodeId", "Sign", "Time"]
    reddit_df = pd.DataFrame(data=reddit_list, columns =col_names)
    # convert the Time from string to datetime format
    reddit_df["Time"] = pd.to_datetime(reddit_df["Time"])
     
    reddit_df.to_pickle(bin_file)

#creation of unique mapping between the subreddit name to a integer.
all_nodes = set(reddit_df.FromNodeId.unique()).union(set(reddit_df.ToNodeId.unique()))
node_int_map = {node: node_id for node_id, node in enumerate(all_nodes)}
reddit_df['FromNodeId'] = reddit_df['FromNodeId'].map(node_int_map)
reddit_df['ToNodeId'] = reddit_df['ToNodeId'].map(node_int_map)

# creation of the graph
reddit_graph = create_graph_from_df(reddit_df)

#### Dataset Statistics

In [25]:
## Counting Nodes and Edges in Reddit Hyperlink Network
reddit_nodes, reddit_edges = count_nodes_edges(reddit_graph)
print("The number of nodes in Reddit Hyperlink Network are {0}".format(reddit_nodes))
print("The number of edges in Reddit Hyperlink Network are {0}".format(reddit_edges))
print()
## Counting the signs of the Edges in Reddit Hyperlink Network
reddit_pos_edges, reddit_neg_edges = count_edge_signs(reddit_graph)
print("The percentage of positive edges in the Reddit Hyperlink Network is {0}".format(reddit_pos_edges))
print("The percentage of negative edges in the Reddit Hyperlink Network is {0}".format(reddit_neg_edges))
print()

The number of nodes in Reddit Hyperlink Network are 35776
The number of edges in Reddit Hyperlink Network are 137821

The percentage of positive edges in the Reddit Hyperlink Network is 94.4
The percentage of negative edges in the Reddit Hyperlink Network is 5.6



#### Analysis of the Reddit Hyperlink Network as an undirected network

In [26]:
# Counting the triads of each type in undirected Reddit Hyperlink Network
reddit_undir_triads = count_undir_triads(reddit_graph, "./data/cache/reddit_undir_triad")

# creating a Reddit Hyperlink Network graph with randomised edge signs
rnd_reddit_graph_df = randomize_df(reddit_df)
rnd_reddit_graph = create_graph_from_df(rnd_reddit_graph_df)

# Counting the triads of each type in undirected Reddit Hyperlink Network with randomised edge signs
rnd_reddit_undir_triads = count_undir_triads(rnd_reddit_graph, "./data/cache/rnd_reddit_undir_triad")

reddit_graph_stats = get_undirected_metrics(reddit_undir_triads, rnd_reddit_undir_triads)

print("The number of Triads in Reddit Hyperlink Network are {0}".format(reddit_graph_stats["delta"]))

The number of Triads in Reddit Hyperlink Network are 406391


#### Analysis of the Reddit Hyperlink Network as an evolving directed network

Using the already created graph, we obtain the generative and recptive baselines for each node.

In [27]:
reddit_genbase, reddit_recbase = get_baselines(reddit_graph)

100%|██████████| 35776/35776 [00:00<00:00, 70138.89it/s]


Get the census realted to the directed graph

In [28]:
reddit_dirgraph_stats, reddit_recpr_dict = get_directed_stats(reddit_df, tri_mapping, reddit_genbase, reddit_recbase)

137821it [00:14, 9557.58it/s]


To calculate the surprise i.e. the number of standard deviations by which the actual number of positive A-B edges for a triad differs above or below the number obtained through the bases we need the prior probability of having a positive edge for each triad type. For this, we shall keep the edge sequence and order the same and randomize the edge sign while maintaining the number of positive and negative signs. This is already obtained while performing the analysis on the undirected graph and we shall the same.

In [29]:
# Get the census realted to the randomised directed graph
# As we only need the number of positive edges for each type of triad and do not bother about other attributes, we use the ols generative and receptive bases
reddit_rnd_dirgraph_stats, _ = get_directed_stats(rnd_reddit_graph_df, tri_mapping, reddit_genbase, reddit_recbase)

137821it [00:14, 9455.23it/s]


Using the information on number of positive edges in each type of triad, obtained by randomised sign graph. We get the priori probability of having a positive edge for each triad type.

In [30]:
reddit_PoTi = get_posprob(reddit_rnd_dirgraph_stats)

# Evaluation with respect to balance and status theories.
reddit_eval = directed_evaluation(reddit_dirgraph_stats, reddit_PoTi)

#### Visualisation and Discussion

In [31]:
print("Reddit Hyperlink Network Dataset Statitics on undirected graph")
print(draw_stats_table(reddit_nodes, reddit_edges, reddit_pos_edges, reddit_neg_edges, reddit_graph_stats["delta"]))
print("\n")
print("Analysis of Undirected Reddit Hyperlink Network")
print(draw_undir_table(reddit_graph_stats))

Reddit Hyperlink Network Dataset Statitics on undirected graph
+----------+--------+
|          | Value  |
+----------+--------+
|  Nodes   | 35776  |
|  Edges   | 137821 |
| +Edges % |  94.4  |
| -Edges % |  5.6   |
|  Triads  | 406391 |
+----------+--------+


Analysis of Undirected Reddit Hyperlink Network
+------------+--------+-------+--------+-------+
| Triad Ti   |  |Ti|  | P(Ti) | Po(Ti) | S(Ti) |
+------------+--------+-------+--------+-------+
| T3 | + + + | 328817 | 0.809 | 0.842  | -57.7 |
| T1 | + - - |  8662  | 0.021 | 0.009  |  86.6 |
| T2 | + + - | 68461  | 0.168 | 0.149  |  34.9 |
| T0 | - - - |  451   | 0.001 |  0.0   |  41.5 |
+------------+--------+-------+--------+-------+


##### Description of undirected analysis results

The triads for this network can be interpreted similarly to friendship.

We find that the all-positive triad $T_3$ is underrepresented, and the triads of type $T_2$, $T_1$ and $T_0$ are overrepresented. This phenomenon cannot be explained by both Heider’s original notion of structural balance and Davis’s weaker notion of balance. 

<hr style="border:1px solid black"> </hr>

In [32]:
print("ti incidates one of the 16 type of triads shown above, P(+) is the porbability that the closing red edge is positive.")
print("sg: surprise of edge initiator giving a positive edge; sr: surprise of edge destination receiving a positive edge") 
print("Bg: consistency of balance with generative surprise; Br: consistency of balance with receptive surprise; \nSg: consistency of status with generative surprise; Sr: consistency of status with receptive surprise.\n")
print("Analysis of the Reddit Hyperlink Network as an evolving directed network\n")
print(draw_dir_table(reddit_eval))

ti incidates one of the 16 type of triads shown above, P(+) is the porbability that the closing red edge is positive.
sg: surprise of edge initiator giving a positive edge; sr: surprise of edge destination receiving a positive edge
Bg: consistency of balance with generative surprise; Br: consistency of balance with receptive surprise; 
Sg: consistency of status with generative surprise; Sr: consistency of status with receptive surprise.

Analysis of the Reddit Hyperlink Network as an evolving directed network

+-----+--------+------+------+-------+-------+-------+-------+-------+
|  ti | count  | P(+) |  sg  |   sr  |   Bg  |   Br  |   Sg  |   Sr  |
+-----+--------+------+------+-------+-------+-------+-------+-------+
|  t1 | 189574 | 0.96 | 95.7 | 131.7 |  True |  True |  True |  True |
|  t2 | 12164  | 0.93 | 18.7 |  37.7 | False | False | False |  True |
|  t3 | 178217 | 0.95 | 85.4 |  27.6 |  True |  True | False |  True |
|  t4 | 11257  | 0.93 | 18.9 |  36.3 | False | False |  Tr

##### Description of on directed analysis results

The triads t1, t3, t9, and t11 are also abundant in this Reddit dataset. These triads with all positive directed edges are only overrepresented.

Except for the previously mentioned 4 triads, the rest of them are underrepresented.

Both the balance and status theory perform similarly on this data. The balance and status theory is consistent on 8 types for the generative surprise and 9 types for the receptive surprise. Taking a look at both generative and receptive surprise together, balance theory is consistent in 8 types where are the status theory is consistent only in 4 types. 

<hr style="border:1px solid black"> </hr>

In [33]:
print("Analysis of edge Reciprocation in directed Reddit Hyperlink Network")
print(draw_recp_table(reddit_recpr_dict))
tot_recp = sum(val for val in reddit_recpr_dict.values())
reddit_recp_perc = (tot_recp/reddit_edges)*100
print("The percentage of edges that reciprocate the edge are {0:.2f}".format(reddit_recp_perc))

Analysis of edge Reciprocation in directed Reddit Hyperlink Network
+---------------+-------+----------+
| Reciprocation | count | Fraction |
+---------------+-------+----------+
|     P(+|+)    | 12847 |   0.98   |
|     P(-|+)    |  270  |   0.02   |
|     P(+|-)    |  345  |   0.92   |
|     P(-|-)    |   29  |   0.08   |
+---------------+-------+----------+
The percentage of edges that reciprocate the edge are 9.79


##### Description of edge reciprocation results

##### TODO: Nikhil


<hr style="border:2px solid black"> </hr>